In [17]:
# File Handling
import h5py

# Signal processing
import numpy as np
import scipy.io as io
import pandas as pd
import scipy.signal 
import sklearn as sk


path_dict = {'DATA.CORE': '/Users/maryam/Desktop/'}
Dataset = pd.read_csv('{}Recent_Contract_Awards.csv'.format(path_dict['DATA.CORE']))

Dataset.head(n=5)
# Start = Dataset["StartDate"]


,RequestID,StartDate,EndDate,AgencyName,TypeOfNoticeDescription,CategoryDescription,ShortTitle,SelectionMethodDescription,SectionName,SpecialCaseReasonDescription,...,Printout2,Printout3,DocumentLinks,EventDate,EventBuildingName,EventStreetAddress1,EventStreetAddress2,EventCity,EventStateCode,EventZipCode
0,20100628021,07/06/2010,07/06/2010,Transportation,Award,Services (other than human services),NOTICE OF AWARD OF CONCESSION AT EAST RIVER PI...,Competitive Sealed Bids,Procurement,NaN,...,NaN,NaN,https://a856-cityrecord.nyc.gov/Search/GetFile...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20130214010,02/22/2013,02/22/2013,Parks and Recreation,Award,Services (other than human services),DEPARTMENT OF PARKS AND RECREATION NOTICE OF...,Public Bid,Procurement,NaN,...,NaN,NaN,https://a856-cityrecord.nyc.gov/Search/GetFile...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20090911004,09/18/2009,09/18/2009,Transportation,Award,Services (other than human services),"OPERATION, MANAGEMENT, AND MAINTENANCE OF PEDE...",Sole Source,Procurement,Other (Describe below in Other Legally Mandate...,...,NaN,NaN,https://a856-cityrecord.nyc.gov/Search/GetFile...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20130415019,04/22/2013,04/22/2013,Parks and Recreation,Award,Goods,NOTICE OF AWARD,Public Bid,Procurement,NaN,...,NaN,NaN,https://a856-cityrecord.nyc.gov/Search/GetFile...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20110623031,06/24/2011,06/24/2011,Transportation,Award,Services (other than human services),AIRPORT BUS FRANCHISE NOTICE OF AWARD,Request for Proposals,Procurement,NaN,...,NaN,NaN,https://a856-cityrecord.nyc.gov/Search/GetFile...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
def IsValidEntry(row):            
        Splits = row["StartDate"].split('/')
#         print(Splits)
        return Splits[2]>'2009' and Splits[2]<'2020' and row["ContractAmount"]>0 
    


In [19]:

ValidIndex = [IsValidEntry(row) for _, row in Dataset.iterrows()]

print(ValidIndex[0:10])
NewDataset = Dataset[ValidIndex]
NewDataset.head(n=5)


[True, True, False, True, True, True, True, False, False, False]


,RequestID,StartDate,EndDate,AgencyName,TypeOfNoticeDescription,CategoryDescription,ShortTitle,SelectionMethodDescription,SectionName,SpecialCaseReasonDescription,...,Printout2,Printout3,DocumentLinks,EventDate,EventBuildingName,EventStreetAddress1,EventStreetAddress2,EventCity,EventStateCode,EventZipCode
0,20100628021,07/06/2010,07/06/2010,Transportation,Award,Services (other than human services),NOTICE OF AWARD OF CONCESSION AT EAST RIVER PI...,Competitive Sealed Bids,Procurement,NaN,...,NaN,NaN,https://a856-cityrecord.nyc.gov/Search/GetFile...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20130214010,02/22/2013,02/22/2013,Parks and Recreation,Award,Services (other than human services),DEPARTMENT OF PARKS AND RECREATION NOTICE OF...,Public Bid,Procurement,NaN,...,NaN,NaN,https://a856-cityrecord.nyc.gov/Search/GetFile...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20130415019,04/22/2013,04/22/2013,Parks and Recreation,Award,Goods,NOTICE OF AWARD,Public Bid,Procurement,NaN,...,NaN,NaN,https://a856-cityrecord.nyc.gov/Search/GetFile...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20110623031,06/24/2011,06/24/2011,Transportation,Award,Services (other than human services),AIRPORT BUS FRANCHISE NOTICE OF AWARD,Request for Proposals,Procurement,NaN,...,NaN,NaN,https://a856-cityrecord.nyc.gov/Search/GetFile...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,20130621002,06/27/2013,06/27/2013,Parks and Recreation,Award,Goods and Services,OPERATION AND MAINTENANCE OF UP TO TWO (2) BOO...,Request for Proposals,Procurement,NaN,...,NaN,NaN,https://a856-cityrecord.nyc.gov/Search/GetFile...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
print(sum(NewDataset["ContractAmount"]))



207783994327.8391


In [51]:
def FindAgencyName(dataF,Name):            
        Agency = dataF["AgencyName"].str.find(Name)
#         print(Agency)
        return Agency>=0



# Question 2 : Determine the number of contracts awarded by each agency. For the top 5 agencies in terms the number of contracts, compute the mean ContractAmount per contract. Among these values, what is the ratio of the highest mean contract amount to the second highest?

In [26]:

UniqueAgency = Dataset["AgencyName"].dropna().unique().tolist() 
MEAN_Contract_Agency = np.zeros(len(UniqueAgency))

print(len(UniqueAgency))
a= 0
for index in UniqueAgency:    
    Ind = FindAgencyName(NewDataset,index)
    ThisFrame = NewDataset[Ind]
    MEAN_Contract_Agency[a] = np.mean(ThisFrame["ContractAmount"])
    a =a +1

SORT = sorted(MEAN_Contract_Agency,reverse = True)
print(SORT[0]/SORT[1])

72
2.6766231552947186


# Question 3 : Citywide Administrative Services agency and compute the sum contract amount awarded to each unique vendor. What proportion of the total number of contracts in the data set were awarded to the top 50 vendors?

In [52]:
Ind_City = FindAgencyName(NewDataset,'Citywide Administrative Services')  

CityDataSet = NewDataset[Ind_City]

CityDataSet.head(n=5)

,RequestID,StartDate,EndDate,AgencyName,TypeOfNoticeDescription,CategoryDescription,ShortTitle,SelectionMethodDescription,SectionName,SpecialCaseReasonDescription,...,Printout2,Printout3,DocumentLinks,EventDate,EventBuildingName,EventStreetAddress1,EventStreetAddress2,EventCity,EventStateCode,EventZipCode
202,20110726010,08/02/2011,08/02/2011,Citywide Administrative Services,Award,Construction/Construction Services,"ELEVATOR UPGRADE, 80 CENTRE STREET",Competitive Sealed Bids,Procurement,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
206,20151019009,10/26/2015,10/26/2015,Citywide Administrative Services,Award,Goods,"EQUIPMENT II, RENTAL OF VARIOUS",Other,Procurement,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
207,20190531014,06/07/2019,06/07/2019,Citywide Administrative Services,Award,Goods,NYS OGS - TASK FORCE RESCUE SYSTEM - FDNY,Intergovernmental Purchase,Procurement,Other (Describe below in Other Legally Mandate...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
209,20160420023,04/27/2016,04/27/2016,Citywide Administrative Services,Award,Goods,SNACKS,Competitive Sealed Bids,Procurement,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
224,20160425024,05/02/2016,05/02/2016,Citywide Administrative Services,Award,Goods,"FRUITS AND VEGETABLES - FRESH, SEASONAL",Competitive Sealed Bids,Procurement,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
def IsValidEntryCount(dataF,Name):            
        Jobs= dataF["VendorName"].str.find(Name)
        return Jobs>=0
    

In [39]:
UniqueVendor = CityDataSet["VendorName"].dropna().unique().tolist() 
Vendor_Count = np.zeros(len(UniqueVendor))

print(len(UniqueVendor))

a= 0
for index in UniqueVendor:    
    Ind = IsValidEntryCount(CityDataSet,index)    
    Vendor_Count[a] = sum(Ind)
    a =a +1
    
SORT = sorted(Vendor_Count,reverse = True)



3154


In [42]:
print("proportion of the total number of contracts in the data set that were awarded to the top 50 vendors: ", sum(SORT[0:49])/sum(Vendor_Count))

proportion of the total number of contracts in the data set that were awarded to the top 50 vendors:  0.1971608832807571


# Question 4 : For this question, consider only contracts with in the categories of Construction Related Services and Construction/Construction Services. The ShortTitle field contains a description of the procured goods/services for each contract. Compute the sum contract amount for contracts whose ShortTitle refer to 'CENTRAL PARK' and for those which refer to 'WASHINGTON SQUARE PARK'. What is the ratio of total construction and contruction-related expenditure for the Central Park contracts compared to the Washington Square Park contracts? Note: you should ensure that 'PARK' appears on its own and not as the beginning of another word.

In [71]:
def FindCatName(dataF,Name):            
        Category1 = dataF["CategoryDescription"].str.find(Name)
#         print(Category1)
        return Category1>=0 



In [115]:
Ind_C1 = FindCatName(NewDataset,'Construction Related Services')  
Data1 = NewDataset[Ind_C1]


Ind_C2 = FindCatName(NewDataset,'Construction/Construction Services')  
Data2 = NewDataset[Ind_C2]

bigdata = pd.concat([Data1,Data2],axis = 0)
BIGDat = bigdata.filter(items=['ShortTitle', 'ContractAmount'])

BIGDat.head(n=15)

,ShortTitle,ContractAmount
199,INSTALLATION AND REPAIR OF SIDEWALK SHEDS BID#...,500000.00
214,"MANUFACTURE, DELIVER OR INSTALL 12,000 BICYCLE...",1934960.00
222,CRO-521 CM: CONSTRUCTION MANAGEMENT SERVICES F...,5458941.00
237,REHABILITATION OF HVAC SYSTEMS AT QUEENS DISTR...,4400000.00
262,EMERGENCY DEMOLITION,672222.00
283,ENGINEERING AND DESIGN SERVICES FOR THE CLOSUR...,7199022.00
289,CONSTRUCTION MANAGEMENT SERVICES FOR MED PROJE...,15000000.00
292,FUNDING AGREEMENT: INSTALLATION OF FDR MEMORIA...,150000.00
336,GI-S-DESIGN-2: TASK ORDER CONTRACT FOR DESIGN ...,3850000.00
348,ON-CALL GENERAL CONSTRUCTION SERVICES,3798234.03


In [117]:
def FindShortTitle(dataF,Name):  
        return  dataF.filter(regex= Name)
         

In [120]:

FindShortTitle(BIGDat ,'CENTRAL PARK')
# Ind = FindShortTitle(bigdata ,'CENTRAL PARK')
# ThisFrame = BIGDat(Ind)

print(sum(BIGDat.ContractAmount))

FindShortTitle(BIGDat ,'WASHINGTON SQUARE PARK')
# Ind = FindShortTitle(bigdata ,'CENTRAL PARK')
# ThisFrame = BIGDat(Ind)

print(sum(BIGDat.ContractAmount))

# MEAN_Contract_Agency[a] = np.mean(ThisFrame["ContractAmount"])


32072614734.35999
32072614734.35999


,RequestID,StartDate,EndDate,AgencyName,TypeOfNoticeDescription,CategoryDescription,ShortTitle,SelectionMethodDescription,SectionName,SpecialCaseReasonDescription,...,Printout2,Printout3,DocumentLinks,EventDate,EventBuildingName,EventStreetAddress1,EventStreetAddress2,EventCity,EventStateCode,EventZipCode
30,20120109007,01/13/2012,01/13/2012,Sanitation,Award,Construction/Construction Services,REQUIREMENTS CONTRACT FOR UPGRADING OF UNDERGR...,Competitive Sealed Bids,Procurement,NaN,...,NaN,NaN,https://a856-cityrecord.nyc.gov/Search/GetFile...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
190,20180726005,08/01/2018,08/01/2018,Environmental Protection,Award,Construction/Construction Services,CRO-536 DES: DESIGN SERVICES FOR CROSS RIVER A...,Request for Proposals,Procurement,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
192,20160922011,09/29/2016,09/29/2016,Correction,Award,Construction/Construction Services,DORM K ELEVATOR REPLACEMENT AT ANNA M. KROSS...,Competitive Sealed Bids,Procurement,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
194,20190116002,01/23/2019,01/23/2019,Housing Preservation and Development,Award,Construction/Construction Services,NON-EMERGENCY DEMOLITION,Competitive Sealed Bids,Procurement,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202,20110726010,08/02/2011,08/02/2011,Citywide Administrative Services,Award,Construction/Construction Services,"ELEVATOR UPGRADE, 80 CENTRE STREET",Competitive Sealed Bids,Procurement,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
